### Data Preparation for self-supervised training

This code creates small patches from large, high resolution satellite and UAV images.

In [47]:
from glob import glob
from PIL import Image
from sklearn.feature_extraction import image
import numpy as np
import imagesize
from pathlib import Path
import os
import shutil
from uuid import uuid4
from tqdm import tqdm

In [48]:
Image.MAX_IMAGE_PIXELS = None 

In [49]:
def crop(im, height, width):
    imgwidth, imgheight = im.size
    rows = np.int(imgheight/height)
    cols = np.int(imgwidth/width)
    for i in range(rows):
        for j in range(cols):
            # print (i,j)
            box = (j*width, i*height, (j+1)*width, (i+1)*height)
            yield im.crop(box)

In [50]:
main_path = '/main/path/to/all/datasets'

In [82]:
def pathches(image, size):
    cropped = crop(image, size, size)
    sv = os.path.join(main_path, i.split('/')[-3] + f'__{size}')
    Path(sv).mkdir(parents=True, exist_ok=True)
    while True:
        try:
            im = next(cropped).resize((500,500))
            if len(np.unique(np.where(np.array(im) == 0)[0])) == 500:
                continue
            else:
                name = str(int(uuid4()))
                im.save(os.path.join(sv, name + '.tif'))
        except StopIteration:
            break

In [54]:
images = glob('path/to/folders/with/iamges/*.tif')

In [57]:
for i in tqdm(images):
    w, h = imagesize.get(i)
    img = Image.open(i)
    pathches(img, 1500)
    pathches(img, 2500)

    if (w or h) > 10000:
        pathches(img, 5000)
                
    if (w or h) > 20000:
        pathches(img, 10000)


100%|██████████| 31/31 [38:51<00:00, 75.20s/it]  


In [91]:
len(glob('/media/lilit/40c6a091-173a-4b33-b34f-05832ea51559/romote_sensing/self_supervised_dataset/*/*'))

95482